---
title: Getting started
toc: true
description: >
    A typical workflow of retrieving some metadata and a table from the API
---

In this tutorial, we go over the sort of interaction a user will usually have
with the package: querying the API for some metadata and a getting a table.

## Making a connection

We begin by importing the `CensusAPI` class from the package and creating an
instance of it. This class is how we connect to the custom table creator API.

In [1]:
from census21api import CensusAPI

api = CensusAPI()

## Defining a query

Now, to query a table, we need three parameters:

- a **population type** (e.g. household reference persons, usual residents)
- an **area type** (e.g. local authorities, Senedd electoral regions)
- some **dimensions** (e.g. english proficiency, economic activity)

Let's say we want the counts for different levels of deprivation between the
sexes at a national level. Deprivation statistics are recorded at the household
level.

So, our parameters are:

- population type: usual residents in households
- area type: national
- dimensions: sex and deprivation

## Finding column names

There are a few columns relating to deprivation in the API.

We could go through the dictionary
`census21api.constants.DIMENSIONS_BY_POPULATION_TYPE` to look for the relevant
ones, but they might not be all that descriptive.

Instead, we can retrieve the information from the API:

In [9]:
population_type = "UR_HH"

metadata = api.query_feature(population_type, "dimensions")
metadata[metadata["id"].str.contains("deprivation")]

,id,label,description,total_count,quality_statement_text,population_type
21,hh_deprivation,Household deprivation (6 categories),The dimensions of deprivation used to classify...,6,Caution should be used in interpreting this va...,UR_HH
22,hh_deprivation_education,Household deprived in the education dimension ...,A household is classified as deprived in the e...,3,,UR_HH
23,hh_deprivation_employment,Household deprived in the employment dimension...,A household is classified as deprived in the e...,3,,UR_HH
24,hh_deprivation_health,Household deprived in the health and disabilit...,A household is classified as deprived in the h...,3,,UR_HH
25,hh_deprivation_housing,Household deprived in the housing dimension (3...,A household is classified as deprived in the h...,3,,UR_HH


After inspecting this data frame, it looks like `hh_deprivation` suits our
purposes - it's an aggregate count of all the deprivation indicators.

## Retrieving a table

With that information, we can use our `CensusAPI` instance to retrieve the
corresponding table from the API using its `query_table()` method:

In [6]:
area_type = "nat"
dimensions = ("sex", "hh_deprivation")

table = api.query_table(population_type, area_type, dimensions, use_id=False)
table

,nat,sex,hh_deprivation,count,population_type
0,England and Wales,Female,Does not apply,0,UR_HH
1,England and Wales,Female,Household is not deprived in any dimension,14536007,UR_HH
2,England and Wales,Female,Household is deprived in one dimension,9928414,UR_HH
3,England and Wales,Female,Household is deprived in two dimensions,4269937,UR_HH
4,England and Wales,Female,Household is deprived in three dimensions,1087182,UR_HH
5,England and Wales,Female,Household is deprived in four dimensions,70177,UR_HH
6,England and Wales,Male,Does not apply,0,UR_HH
7,England and Wales,Male,Household is not deprived in any dimension,14375435,UR_HH
8,England and Wales,Male,Household is deprived in one dimension,9265887,UR_HH
9,England and Wales,Male,Household is deprived in two dimensions,3854022,UR_HH


And we're done - we have retrieved our table and we can go on to analyse it in
whichever way we see fit.